In [ ]:
import numpy as np
from pydrake.all import (
    DiagramBuilder,
    JointIndex,
    LeafSystem,
    MeshcatPoseSliders,
    QuaternionFloatingJoint,
    RigidTransform,
    Role,
    Simulator,
    StartMeshcat,
)

from manipulation.scenarios import AddIiwaDifferentialIK, ExtractBodyPose
from manipulation.station import MakeHardwareStation, load_scenario
from manipulation.utils import running_as_notebook

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# Suction example

A simple teleop example w/ the simplest model of "suction", implemented in a temporary way by adding floating joints between the gripper and the floating objects, and locking/unlocking those joints.

Note: This should get much cleaner pending the resolution of https://github.com/RobotLocomotion/drake/issues/20571.

In [ ]:
scenario_data = """
directives:
- add_model:
    name: iiwa
    file: package://manipulation/planar_iiwa14_no_collision.urdf
    default_joint_positions:
        iiwa_joint_2: [0.1]
        iiwa_joint_4: [-1.2]
        iiwa_joint_6: [1.6]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: suction_base
    file: package://manipulation/suction/suction-base.urdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: suction_base::baseLink
    X_PC:
        translation: [0, 0, 0.01]
        rotation: !Rpy { deg: [0, 0, 0]}
- add_model:
    name: suction_head
    file: package://manipulation/suction/suction-head.urdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: suction_head::headLink
    X_PC:
        translation: [0, 0, 0.10]
        rotation: !Rpy { deg: [0, 0, 0]}

- add_model:
    name: brick0
    file: package://manipulation/hydro/061_foam_brick.sdf

- add_model:
    name: brick1
    file: package://manipulation/hydro/061_foam_brick.sdf

- add_model:
    name: robot_table
    file: package://manipulation/hydro/extra_heavy_duty_table_surface_only_collision.sdf
- add_weld:
    parent: world
    child: robot_table::link
    X_PC:
        translation: [0, 0, -0.7645]
- add_model:
    name: work_table
    file: package://manipulation/hydro/extra_heavy_duty_table_surface_only_collision.sdf
- add_weld:
    parent: world
    child: work_table::link
    X_PC:
        translation: [0.75, 0, -0.7645]

model_drivers:
    iiwa: !IiwaDriver {}
"""

# TODO(russt): Implement free joints in urdf, and replace the add_joint callback
# with the following model_directives:
# - add_model:
#    name: brick0_floating
#    file: package://manipulation/floating_joint.urdf
#    default_joint_positions:
#        floating: [1, 0, 0, 0, 0, 0, 3]  # quat, x, y, z
# - add_weld:
#    parent: suction_head::tipLink
#    child: brick0_floating::parent
# - add_weld:
#    parent: brick0_floating::child
#    child: brick0::base_link


class SuctionButton(LeafSystem):
    def __init__(self, meshcat):
        LeafSystem.__init__(self)
        port = self.DeclareVectorOutputPort("suction_on", 1, self.DoCalcOutput)
        port.disable_caching_by_default()
        self._meshcat = meshcat
        self._button = "Suction on/off"
        meshcat.AddButton(self._button, "Space")
        print("Press Space to turn the suction on/off")

    def __del__(self):
        self._meshcat.DeleteButton(self._button)

    def DoCalcOutput(self, context, output):
        output.SetAtIndex(0, self._meshcat.GetButtonClicks(self._button) % 2)


def teleop_2d():
    scenario = load_scenario(data=scenario_data)
    meshcat.Set2dRenderMode(xmin=-0.25, xmax=1.5, ymin=-0.1, ymax=1.3)

    builder = DiagramBuilder()

    def add_suction_joints(parser):
        plant = parser.plant()
        suction_body = plant.GetBodyByName("tipLink")
        brick0_id = plant.GetModelInstanceByName("brick0")
        brick0_body = plant.get_body(plant.GetBodyIndices(brick0_id)[0])
        brick0_joint = plant.AddJoint(
            QuaternionFloatingJoint(
                "brick0_joint",
                suction_body.body_frame(),
                brick0_body.body_frame(),
            )
        )
        # Inconveniently, this is in the frame of the suction gripper, and you
        # can't evaluate the kinematics of this plant yet. In a pinch, make a
        # second iiwa here and use that to compute the kinematics, then throw
        # it away.
        X_SuctionBrick0 = RigidTransform([0, 0, 0.2])
        brick0_joint.SetDefaultPose(X_SuctionBrick0)

        brick1_id = plant.GetModelInstanceByName("brick1")
        brick1_body = plant.get_body(plant.GetBodyIndices(brick1_id)[0])
        brick1_joint = plant.AddJoint(
            QuaternionFloatingJoint(
                "brick1_joint",
                suction_body.body_frame(),
                brick1_body.body_frame(),
            )
        )
        X_SuctionBrick1 = RigidTransform([0, 0.2, 0.2])
        brick1_joint.SetDefaultPose(X_SuctionBrick1)

    station = builder.AddSystem(
        MakeHardwareStation(
            scenario, meshcat, parser_prefinalize_callback=add_suction_joints
        )
    )

    # TODO(russt): Move this whole block into station.AddDiffIk(scenario, frame, builder)
    # Set up differential inverse kinematics.
    controller_plant = station.GetSubsystemByName(
        "iiwa.controller"
    ).get_multibody_plant_for_control()
    differential_ik = AddIiwaDifferentialIK(
        builder,
        controller_plant,
        frame=controller_plant.GetFrameByName("iiwa_link_7"),
    )
    builder.Connect(
        differential_ik.get_output_port(),
        station.GetInputPort("iiwa.position"),
    )
    builder.Connect(
        station.GetOutputPort("iiwa.state_estimated"),
        differential_ik.GetInputPort("robot_state"),
    )

    # Set up teleop widgets.
    meshcat.DeleteAddedControls()
    teleop = builder.AddSystem(
        MeshcatPoseSliders(
            meshcat,
            lower_limit=[0, -np.pi, -np.pi, -0.6, -1, 0],
            upper_limit=[2 * np.pi, np.pi, np.pi, 0.8, 1, 1.1],
            # Only roll, x, and z are used in this example:
            visible=[True, False, False, True, False, True],
            decrement_keycodes=["KeyQ", "", "", "ArrowLeft", "", "ArrowDown"],
            increment_keycodes=["KeyE", "", "", "ArrowRight", "", "ArrowUp"],
        )
    )
    builder.Connect(
        teleop.get_output_port(), differential_ik.GetInputPort("X_WE_desired")
    )
    # Note: This is using "Cheat Ports". For it to work on hardware, we would
    # need to construct the initial pose from the HardwareStation outputs.
    plant = station.GetSubsystemByName("plant")
    ee_pose = builder.AddSystem(
        ExtractBodyPose(
            station.GetOutputPort("body_poses"),
            plant.GetBodyByName("iiwa_link_7").index(),
        )
    )
    builder.Connect(station.GetOutputPort("body_poses"), ee_pose.get_input_port())
    builder.Connect(ee_pose.get_output_port(), teleop.get_input_port())

    suction_button = builder.AddSystem(SuctionButton(meshcat))

    # Simulate.
    diagram = builder.Build()
    simulator = Simulator(diagram)

    context = simulator.get_context()
    plant_context = plant.GetMyContextFromRoot(context)
    scene_graph = station.GetSubsystemByName("scene_graph")
    scene_graph_context = scene_graph.GetMyContextFromRoot(context)
    tip_geom_id = scene_graph.model_inspector().GetGeometries(
        plant.GetBodyFrameIdOrThrow(plant.GetBodyByName("tipLink").index()),
        Role.kProximity,
    )[0]

    suction_context = suction_button.GetMyContextFromRoot(context)
    suction_max_distance = 0.01
    last_suction_state = False
    active_suction_joints = []

    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)
        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            suction_state = suction_button.GetOutputPort("suction_on").Eval(
                suction_context
            )
            if suction_state and not last_suction_state:
                print("turning suction on")
                query_object = scene_graph.get_query_output_port().Eval(
                    scene_graph_context
                )
                inspector = query_object.inspector()
                for pair in query_object.ComputeSignedDistancePairwiseClosestPoints(
                    suction_max_distance
                ):
                    # Do a little gymnastics to figure out which joint to lock
                    suctioned_geom_id = None
                    if pair.id_A == tip_geom_id:
                        suctioned_geom_id = pair.id_B
                    elif pair.id_B == tip_geom_id:
                        suctioned_geom_id.geom_id = pair.id_A
                    else:
                        continue
                    suctioned_frame_id = inspector.GetFrameId(suctioned_geom_id)
                    suctioned_body = plant.GetBodyFromFrameId(suctioned_frame_id)

                    for i in range(plant.num_joints()):
                        joint = plant.get_joint(JointIndex(i))
                        if joint.child_body() == suctioned_body:
                            joint.Lock(plant_context)
                            active_suction_joints.append(joint)
                            print(f"suctioned the {suctioned_body.name()}")
                if not active_suction_joints:
                    print("too far away to suction anything")

            elif not suction_state and last_suction_state:
                print("turning suction off")
                for joint in active_suction_joints:
                    joint.Unlock(plant_context)
                active_suction_joints = []

            last_suction_state = suction_state
            simulator.AdvanceTo(context.get_time() + 0.1)

        meshcat.DeleteButton("Stop Simulation")
    else:
        simulator.set_target_realtime_rate(0)
        simulator.AdvanceTo(0.1)


teleop_2d()